# Reusable Spark Libraries -- Missing Data Imputation

#### Handling missing data is a big part of our job, so a consistent, common set of tools to handle it is important.

### Imports & Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

from src.main.imputation.simple_imputer import SimpleImputer
from src.main.imputation.mean_imputer import MeanImputer
from src.main.imputation.median_imputer import MedianImputer
from src.main.imputation.mode_imputer import ModeImputer
from src.main.imputation.forward_fill_imputer import ForwardFillImputer
from src.main.imputation.backward_fill_imputer import BackwardFillImputer
from src.main.imputation.linear_regression_imputer import LinearRegressionImputer


In [3]:
spark = SparkSession \
                    .builder \
                    .master('local') \
                    .appName("Imputation Demo") \
                    .getOrCreate()

In [4]:
ts_data_path = "../datasets/demo_data.csv"

data_ts = pd.read_csv(ts_data_path)

schema_ts = StructType([
    StructField("CustomerId", IntegerType()),
    StructField("AccountType", StringType()),
    StructField("Country", StringType()),
    StructField("Date", StringType()),
    StructField("Balances", FloatType()),
    StructField("MaxAccountBalance", IntegerType()),
    StructField("EliteMember", StringType())
])

df_ts = spark \
    .createDataFrame(data=data_ts, schema=schema_ts) \
    .withColumn("Date", F.to_date(F.col("Date"), "yyyy-mm-dd")) \
    .replace(float("nan"), None) \
    .replace("NaN", None)


In [5]:
df_ts.show()

+----------+-----------+-------+----------+--------+-----------------+-----------+
|CustomerId|AccountType|Country|      Date|Balances|MaxAccountBalance|EliteMember|
+----------+-----------+-------+----------+--------+-----------------+-----------+
|    882704|    Current|     CA|2019-01-01|    null|          7143141|        Yes|
|     92929|    Savings|     US|2019-01-01|221680.0|          6554191|         No|
|    625741|    Savings|     CA|2019-01-01|560797.0|          2731430|       null|
|    711485|    Current|     CA|2019-01-01| 29739.0|          5973700|         No|
|    882704|    Current|     CA|2019-01-02| 85406.0|          7143141|        Yes|
|     92929|    Savings|     US|2019-01-02|695791.0|          6554191|         No|
|    625741|    Savings|     CA|2019-01-02|740800.0|          2731430|       null|
|    711485|    Current|     CA|2019-01-02| 22494.0|          5973700|         No|
|    882704|    Current|     CA|2019-01-03|    null|          7143141|        Yes|
|   

# Imputation Demo

## Simple Imputer


#### Well-documented, easy to understand.

All imputation classes are well-documented with all pertinent information easily available.



In [6]:
print(SimpleImputer.__doc__)


    A class that acts as a wrapper for Mean, Median, Mode, Forward/Backward Fill imputation in their most simple form.

    For more nuanced usage of the methods, use the individual imputers themselves.

    Parameters
    ----------
    strategy : str
        The single imputation method to use in its most simple form.
        Defaults to `mean` if not specified.
        Currently, `mean`, `median`, `mode`, `ffill`, and `bfill` are supported.
        Any imputation method that can map `input_cols`[i] -> `output_cols`[i] can be implemented.
    input_cols : List[str]
        A list of column names with missing values to be imputed.
    output_cols : List[str]
        A list of column names where for index i in 0, ..., n-1 for
        input_cols of length n, input_cols[i] -> output_cols[i].
        Defaults to equaling input_cols and overwrites the input columns.
    missing_value : float
        A specified value to be imputed in addition to `null`.
    relative_error : float
        

#### Basic usage

Simple Imputer defaults to using `mean` to impute a column.

In [7]:
simple_imputer = SimpleImputer(input_cols=["Balances"])

simple_imputer.impute(df_ts).show()

+----------+-----------+-------+----------+---------+-----------------+-----------+
|CustomerId|AccountType|Country|      Date| Balances|MaxAccountBalance|EliteMember|
+----------+-----------+-------+----------+---------+-----------------+-----------+
|    882704|    Current|     CA|2019-01-01|312963.12|          7143141|        Yes|
|     92929|    Savings|     US|2019-01-01| 221680.0|          6554191|         No|
|    625741|    Savings|     CA|2019-01-01| 560797.0|          2731430|       null|
|    711485|    Current|     CA|2019-01-01|  29739.0|          5973700|         No|
|    882704|    Current|     CA|2019-01-02|  85406.0|          7143141|        Yes|
|     92929|    Savings|     US|2019-01-02| 695791.0|          6554191|         No|
|    625741|    Savings|     CA|2019-01-02| 740800.0|          2731430|       null|
|    711485|    Current|     CA|2019-01-02|  22494.0|          5973700|         No|
|    882704|    Current|     CA|2019-01-03|312963.12|          7143141|     

We can specify output columns to preserve original columns

In [8]:
simple_imputer.output_cols = ["BalancesImputed"]

simple_imputer.impute(df_ts).show()

+----------+-----------+-------+----------+--------+-----------------+-----------+---------------+
|CustomerId|AccountType|Country|      Date|Balances|MaxAccountBalance|EliteMember|BalancesImputed|
+----------+-----------+-------+----------+--------+-----------------+-----------+---------------+
|    882704|    Current|     CA|2019-01-01|    null|          7143141|        Yes|      312963.12|
|     92929|    Savings|     US|2019-01-01|221680.0|          6554191|         No|       221680.0|
|    625741|    Savings|     CA|2019-01-01|560797.0|          2731430|       null|       560797.0|
|    711485|    Current|     CA|2019-01-01| 29739.0|          5973700|         No|        29739.0|
|    882704|    Current|     CA|2019-01-02| 85406.0|          7143141|        Yes|        85406.0|
|     92929|    Savings|     US|2019-01-02|695791.0|          6554191|         No|       695791.0|
|    625741|    Savings|     CA|2019-01-02|740800.0|          2731430|       null|       740800.0|
|    71148

#### Changing strategies

Currently, `mean`, `median`, `mode`, `ffill`, `bfill` are supported.

We only have to change the strategy, and add an `order_by_cols` param since we're working on the same column.

In [9]:
simple_imputer.strategy = "ffill"
simple_imputer.order_by_cols = ["Date"]

simple_imputer.impute(df_ts).show()

+----------+-----------+-------+----------+--------+-----------------+-----------+---------------+
|CustomerId|AccountType|Country|      Date|Balances|MaxAccountBalance|EliteMember|BalancesImputed|
+----------+-----------+-------+----------+--------+-----------------+-----------+---------------+
|    882704|    Current|     CA|2019-01-01|    null|          7143141|        Yes|           null|
|     92929|    Savings|     US|2019-01-01|221680.0|          6554191|         No|       221680.0|
|    625741|    Savings|     CA|2019-01-01|560797.0|          2731430|       null|       560797.0|
|    711485|    Current|     CA|2019-01-01| 29739.0|          5973700|         No|        29739.0|
|    882704|    Current|     CA|2019-01-02| 85406.0|          7143141|        Yes|        85406.0|
|     92929|    Savings|     US|2019-01-02|695791.0|          6554191|         No|       695791.0|
|    625741|    Savings|     CA|2019-01-02|740800.0|          2731430|       null|       740800.0|
|    71148

Only need to change strategy now, since `order_by_cols` is already specified.

In [10]:
simple_imputer.strategy = "bfill"

simple_imputer.impute(df_ts).show()

+----------+-----------+-------+----------+--------+-----------------+-----------+---------------+
|CustomerId|AccountType|Country|      Date|Balances|MaxAccountBalance|EliteMember|BalancesImputed|
+----------+-----------+-------+----------+--------+-----------------+-----------+---------------+
|    882704|    Current|     CA|2019-01-01|    null|          7143141|        Yes|       221680.0|
|     92929|    Savings|     US|2019-01-01|221680.0|          6554191|         No|       221680.0|
|    625741|    Savings|     CA|2019-01-01|560797.0|          2731430|       null|       560797.0|
|    711485|    Current|     CA|2019-01-01| 29739.0|          5973700|         No|        29739.0|
|    882704|    Current|     CA|2019-01-02| 85406.0|          7143141|        Yes|        85406.0|
|     92929|    Savings|     US|2019-01-02|695791.0|          6554191|         No|       695791.0|
|    625741|    Savings|     CA|2019-01-02|740800.0|          2731430|       null|       740800.0|
|    71148

Don't need to remove the `order_by_cols` parameter, since `mode` does not use it.

In [11]:
simple_imputer.strategy = "mode"
simple_imputer.input_cols = ["EliteMember"]
simple_imputer.output_cols = ["EliteMemberImputed"]

simple_imputer.impute(df_ts).show()

+----------+-----------+-------+----------+--------+-----------------+-----------+------------------+
|CustomerId|AccountType|Country|      Date|Balances|MaxAccountBalance|EliteMember|EliteMemberImputed|
+----------+-----------+-------+----------+--------+-----------------+-----------+------------------+
|    882704|    Current|     CA|2019-01-01|    null|          7143141|        Yes|               Yes|
|     92929|    Savings|     US|2019-01-01|221680.0|          6554191|         No|                No|
|    625741|    Savings|     CA|2019-01-01|560797.0|          2731430|       null|                No|
|    711485|    Current|     CA|2019-01-01| 29739.0|          5973700|         No|                No|
|    882704|    Current|     CA|2019-01-02| 85406.0|          7143141|        Yes|               Yes|
|     92929|    Savings|     US|2019-01-02|695791.0|          6554191|         No|                No|
|    625741|    Savings|     CA|2019-01-02|740800.0|          2731430|       null|

#### Limitations

`SimpleImputer` is great for quick applications of basic imputation techniques.

However, it fails to perform well when there is a need for more nuanced domain knowledge to be applied.

### Mean Imputation

In [12]:
df_ts.show()

+----------+-----------+-------+----------+--------+-----------------+-----------+
|CustomerId|AccountType|Country|      Date|Balances|MaxAccountBalance|EliteMember|
+----------+-----------+-------+----------+--------+-----------------+-----------+
|    882704|    Current|     CA|2019-01-01|    null|          7143141|        Yes|
|     92929|    Savings|     US|2019-01-01|221680.0|          6554191|         No|
|    625741|    Savings|     CA|2019-01-01|560797.0|          2731430|       null|
|    711485|    Current|     CA|2019-01-01| 29739.0|          5973700|         No|
|    882704|    Current|     CA|2019-01-02| 85406.0|          7143141|        Yes|
|     92929|    Savings|     US|2019-01-02|695791.0|          6554191|         No|
|    625741|    Savings|     CA|2019-01-02|740800.0|          2731430|       null|
|    711485|    Current|     CA|2019-01-02| 22494.0|          5973700|         No|
|    882704|    Current|     CA|2019-01-03|    null|          7143141|        Yes|
|   

#### Without grouping

In [13]:
mean_imputer = MeanImputer(input_cols=["Balances"], output_cols=["BalanceImputed"])

mean_imputer.impute(df_ts).show()

+----------+-----------+-------+----------+--------+-----------------+-----------+--------------+
|CustomerId|AccountType|Country|      Date|Balances|MaxAccountBalance|EliteMember|BalanceImputed|
+----------+-----------+-------+----------+--------+-----------------+-----------+--------------+
|    882704|    Current|     CA|2019-01-01|    null|          7143141|        Yes|     312963.12|
|     92929|    Savings|     US|2019-01-01|221680.0|          6554191|         No|      221680.0|
|    625741|    Savings|     CA|2019-01-01|560797.0|          2731430|       null|      560797.0|
|    711485|    Current|     CA|2019-01-01| 29739.0|          5973700|         No|       29739.0|
|    882704|    Current|     CA|2019-01-02| 85406.0|          7143141|        Yes|       85406.0|
|     92929|    Savings|     US|2019-01-02|695791.0|          6554191|         No|      695791.0|
|    625741|    Savings|     CA|2019-01-02|740800.0|          2731430|       null|      740800.0|
|    711485|    Curr

#### With grouping

Balances are imputed by type of account

In [14]:
mean_imputer.list_group_by_cols = [["AccountType"]]

mean_imputer.impute(df_ts).show()

+-----------+----------+-------+----------+--------+-----------------+-----------+-----------------+
|AccountType|CustomerId|Country|      Date|Balances|MaxAccountBalance|EliteMember|   BalanceImputed|
+-----------+----------+-------+----------+--------+-----------------+-----------+-----------------+
|    Savings|     92929|     US|2019-01-01|221680.0|          6554191|         No|         221680.0|
|    Savings|    625741|     CA|2019-01-01|560797.0|          2731430|       null|         560797.0|
|    Savings|     92929|     US|2019-01-02|695791.0|          6554191|         No|         695791.0|
|    Savings|    625741|     CA|2019-01-02|740800.0|          2731430|       null|         740800.0|
|    Savings|     92929|     US|2019-01-03|    null|          6554191|         No|        540005.25|
|    Savings|    625741|     CA|2019-01-03|    null|          2731430|       null|        540005.25|
|    Savings|     92929|     US|2019-01-04|669368.0|          6554191|         No|         

### Median Imputation

In [15]:
data_median = [
    ("Cashier", 30000.0, 1250.0),
    ("Cashier", None, 750.0),
    ("Cashier", 37000.0, None),
    ("Cashier", 75000.0, 5000.0),
    ("Data Scientist", 75000.0, None),
    ("Data Scientist", 95000.0, 10000.0),
    ("Data Scientist", None, 6500.0),
    ("Data Scientist", 205000.0, 35000.0)
]

df_median = spark.createDataFrame(data_median, ["Occupation", "Salary", "Bonus"])

In [16]:
df_median.show()

+--------------+--------+-------+
|    Occupation|  Salary|  Bonus|
+--------------+--------+-------+
|       Cashier| 30000.0| 1250.0|
|       Cashier|    null|  750.0|
|       Cashier| 37000.0|   null|
|       Cashier| 75000.0| 5000.0|
|Data Scientist| 75000.0|   null|
|Data Scientist| 95000.0|10000.0|
|Data Scientist|    null| 6500.0|
|Data Scientist|205000.0|35000.0|
+--------------+--------+-------+



#### Without grouping

In [17]:
median_imputer = MedianImputer(input_cols=["Salary", "Bonus"])

df_median.show()

median_imputer.impute(df_median).show()

+--------------+--------+-------+
|    Occupation|  Salary|  Bonus|
+--------------+--------+-------+
|       Cashier| 30000.0| 1250.0|
|       Cashier|    null|  750.0|
|       Cashier| 37000.0|   null|
|       Cashier| 75000.0| 5000.0|
|Data Scientist| 75000.0|   null|
|Data Scientist| 95000.0|10000.0|
|Data Scientist|    null| 6500.0|
|Data Scientist|205000.0|35000.0|
+--------------+--------+-------+

+--------------+--------+-------+
|    Occupation|  Salary|  Bonus|
+--------------+--------+-------+
|       Cashier| 30000.0| 1250.0|
|       Cashier| 75000.0|  750.0|
|       Cashier| 37000.0| 5000.0|
|       Cashier| 75000.0| 5000.0|
|Data Scientist| 75000.0| 5000.0|
|Data Scientist| 95000.0|10000.0|
|Data Scientist| 75000.0| 6500.0|
|Data Scientist|205000.0|35000.0|
+--------------+--------+-------+



#### With grouping

In [18]:
median_imputer.list_group_by_cols = [["Occupation"], ["Occupation"]]

df_median.show()

median_imputer.impute(df_median).show()

+--------------+--------+-------+
|    Occupation|  Salary|  Bonus|
+--------------+--------+-------+
|       Cashier| 30000.0| 1250.0|
|       Cashier|    null|  750.0|
|       Cashier| 37000.0|   null|
|       Cashier| 75000.0| 5000.0|
|Data Scientist| 75000.0|   null|
|Data Scientist| 95000.0|10000.0|
|Data Scientist|    null| 6500.0|
|Data Scientist|205000.0|35000.0|
+--------------+--------+-------+

+--------------+--------+-------+
|    Occupation|  Salary|  Bonus|
+--------------+--------+-------+
|       Cashier| 30000.0| 1250.0|
|       Cashier| 37000.0|  750.0|
|       Cashier| 37000.0| 1250.0|
|       Cashier| 75000.0| 5000.0|
|Data Scientist| 75000.0|10000.0|
|Data Scientist| 95000.0|10000.0|
|Data Scientist| 95000.0| 6500.0|
|Data Scientist|205000.0|35000.0|
+--------------+--------+-------+



### Forward / Backward Fill Imputation

In [19]:
df_ts.show()

+----------+-----------+-------+----------+--------+-----------------+-----------+
|CustomerId|AccountType|Country|      Date|Balances|MaxAccountBalance|EliteMember|
+----------+-----------+-------+----------+--------+-----------------+-----------+
|    882704|    Current|     CA|2019-01-01|    null|          7143141|        Yes|
|     92929|    Savings|     US|2019-01-01|221680.0|          6554191|         No|
|    625741|    Savings|     CA|2019-01-01|560797.0|          2731430|       null|
|    711485|    Current|     CA|2019-01-01| 29739.0|          5973700|         No|
|    882704|    Current|     CA|2019-01-02| 85406.0|          7143141|        Yes|
|     92929|    Savings|     US|2019-01-02|695791.0|          6554191|         No|
|    625741|    Savings|     CA|2019-01-02|740800.0|          2731430|       null|
|    711485|    Current|     CA|2019-01-02| 22494.0|          5973700|         No|
|    882704|    Current|     CA|2019-01-03|    null|          7143141|        Yes|
|   

#### Very powerful when order by column has repeated entries 

In [20]:
bfill_imputer = BackwardFillImputer(input_cols=["Balances"],
                                    output_cols=["BalancesImputed"],
                                    order_by_cols=["Date"],
                                    list_partition_by_cols=[["CustomerId"]])

bfill_imputer \
    .impute(df_ts) \
    .orderBy("Date", "CustomerId") \
    .show()

+----------+-----------+-------+----------+--------+-----------------+-----------+---------------+
|CustomerId|AccountType|Country|      Date|Balances|MaxAccountBalance|EliteMember|BalancesImputed|
+----------+-----------+-------+----------+--------+-----------------+-----------+---------------+
|     92929|    Savings|     US|2019-01-01|221680.0|          6554191|         No|       221680.0|
|    625741|    Savings|     CA|2019-01-01|560797.0|          2731430|       null|       560797.0|
|    711485|    Current|     CA|2019-01-01| 29739.0|          5973700|         No|        29739.0|
|    882704|    Current|     CA|2019-01-01|    null|          7143141|        Yes|        85406.0|
|     92929|    Savings|     US|2019-01-02|695791.0|          6554191|         No|       695791.0|
|    625741|    Savings|     CA|2019-01-02|740800.0|          2731430|       null|       740800.0|
|    711485|    Current|     CA|2019-01-02| 22494.0|          5973700|         No|        22494.0|
|    88270

### Linear Regression Imputation

In [21]:
from pyspark.ml.feature import VectorAssembler

lr_data = [
    (1, 651, 23),
    (2, 762, 26),
    (3, 856, 30),
    (4, 1063, None),
    (5, 1190, 43),
    (6, 1298, 48),
    (7, 1421, None),
    (8, 1440, 57),
    (9, 1518, 58)
]

df_lr = spark.createDataFrame(lr_data, ["Year", "Sales (millions USD)", "Advertising (millions USD)"])

In [22]:
df_lr.show()

+----+--------------------+--------------------------+
|Year|Sales (millions USD)|Advertising (millions USD)|
+----+--------------------+--------------------------+
|   1|                 651|                        23|
|   2|                 762|                        26|
|   3|                 856|                        30|
|   4|                1063|                      null|
|   5|                1190|                        43|
|   6|                1298|                        48|
|   7|                1421|                      null|
|   8|                1440|                        57|
|   9|                1518|                        58|
+----+--------------------+--------------------------+



In [23]:
assembler = VectorAssembler(inputCols=["Year", "Sales (millions USD)"], outputCol="features")
df_lr = assembler.transform(df_lr)

df_lr.show()

+----+--------------------+--------------------------+------------+
|Year|Sales (millions USD)|Advertising (millions USD)|    features|
+----+--------------------+--------------------------+------------+
|   1|                 651|                        23| [1.0,651.0]|
|   2|                 762|                        26| [2.0,762.0]|
|   3|                 856|                        30| [3.0,856.0]|
|   4|                1063|                      null|[4.0,1063.0]|
|   5|                1190|                        43|[5.0,1190.0]|
|   6|                1298|                        48|[6.0,1298.0]|
|   7|                1421|                      null|[7.0,1421.0]|
|   8|                1440|                        57|[8.0,1440.0]|
|   9|                1518|                        58|[9.0,1518.0]|
+----+--------------------+--------------------------+------------+



In [24]:
lr_imputer = LinearRegressionImputer(features_col="features",
                                     col_to_impute="Advertising (millions USD)")

lr_imputer.impute(df_lr).show()

+----+--------------------+--------------------------+------------+
|Year|Sales (millions USD)|Advertising (millions USD)|    features|
+----+--------------------+--------------------------+------------+
|   1|                 651|                      23.0| [1.0,651.0]|
|   2|                 762|                      26.0| [2.0,762.0]|
|   3|                 856|                      30.0| [3.0,856.0]|
|   4|                1063|                   36.8376|[4.0,1063.0]|
|   5|                1190|                      43.0|[5.0,1190.0]|
|   6|                1298|                      48.0|[6.0,1298.0]|
|   7|                1421|                 51.715458|[7.0,1421.0]|
|   8|                1440|                      57.0|[8.0,1440.0]|
|   9|                1518|                      58.0|[9.0,1518.0]|
+----+--------------------+--------------------------+------------+

